# Extract accessions from Table S1

In [ ]:
import pandas as pd

df = (
    pd.read_excel("Table_S1.xlsx", sheet_name="Early SARS-CoV-2 genomes")
    .rename(columns={"Genome name": "name"})
    .assign(
        strain=lambda x: x["name"].str.split("|").str[0],
        rest=lambda x: x["name"].str.split("|", n=1).str[1],
        source=lambda x: x["rest"].map(
            lambda s: (
                "custom" if "|" not in s else
                ("gisaid" if s.startswith("EPI") else ("ngdc" if s.startswith("NMDC") else "genbank"))
            )
        ),
        accession=lambda x: x["rest"].str.split("|").str[0].where(x["source"] != "custom", pd.NA),
        date=lambda x: pd.to_datetime(x["rest"].str.split("|").str[-1]),
    )
    .drop(columns="rest")
)

df.to_csv("seq_metadata.csv", index=False)

for source, source_df in df.groupby("source"):
    source_df.to_csv(f"{source}_accessions.csv", index=False)